In [1]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import torch.nn as nn
import numpy as np
import joblib
# -----------------------------
# LSTM Model
# -----------------------------
class AQILSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 64, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

FEATURES = [
    'PM2.5','PM10','NO','NO2','NOx','NH3',
    'CO','SO2','O3','Benzene','Toluene','Xylene'
]

SEQ_LEN = 7

# Load model & scalers
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

model = AQILSTM(len(FEATURES))
model.load_state_dict(torch.load("lstm_aqi_model.pth", map_location="cpu"))
model.eval()

app = FastAPI(title="AQI Prediction API")

# -----------------------------
# Input schema
# -----------------------------
class AQIInput(BaseModel):
    data: list   # list of last 7 days pollutant values

# -----------------------------
# Prediction endpoint
# -----------------------------
@app.post("/predict")
def predict_aqi(input: AQIInput):
    arr = np.array(input.data)
    arr_scaled = scaler_X.transform(arr)

    X = torch.tensor(arr_scaled, dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        pred_scaled = model(X).numpy()

    pred = scaler_y.inverse_transform(pred_scaled)[0][0]

    if pred <= 50:
        category = "Good"
    elif pred <= 100:
        category = "Satisfactory"
    elif pred <= 200:
        category = "Moderate"
    elif pred <= 300:
        category = "Poor"
    elif pred <= 400:
        category = "Very Poor"
    else:
        category = "Severe"

    return {
        "AQI": int(pred),
        "Category": category
    }

Writing app.py


In [2]:
!uvicorn app:app.

^C
